<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#bam2feat.py" data-toc-modified-id="bam2feat.py-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>bam2feat.py</a></span></li><li><span><a href="#bam2feat" data-toc-modified-id="bam2feat-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>bam2feat</a></span></li></ul></li><li><span><a href="#Compare" data-toc-modified-id="Compare-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compare</a></span><ul class="toc-item"><li><span><a href="#Check-that-dataset-matches" data-toc-modified-id="Check-that-dataset-matches-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Check that dataset matches</a></span></li><li><span><a href="#Features" data-toc-modified-id="Features-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Features</a></span><ul class="toc-item"><li><span><a href="#ref-bases" data-toc-modified-id="ref-bases-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>ref bases</a></span></li><li><span><a href="#coverage/SNPs" data-toc-modified-id="coverage/SNPs-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>coverage/SNPs</a></span></li><li><span><a href="#Match-features" data-toc-modified-id="Match-features-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Match features</a></span></li><li><span><a href="#SNP-features" data-toc-modified-id="SNP-features-5.2.4"><span class="toc-item-num">5.2.4&nbsp;&nbsp;</span>SNP features</a></span></li><li><span><a href="#G+C-&amp;-entropy" data-toc-modified-id="G+C-&amp;-entropy-5.2.5"><span class="toc-item-num">5.2.5&nbsp;&nbsp;</span>G+C &amp; entropy</a></span></li></ul></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Check that `bam2feat` works as `bam2feat.py`
  * `bam2feat` is a much faster implementation of `bam2feat.py`

# Var

In [1]:
work_dir = '/ebio/abt3_projects/DeepMAsED/version2/deepmased/DeepMAsED-SM/tests/output_n10/'

threads = 8

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘tidytable’


The following object is masked from ‘package:stats’:

    dt




In [3]:
df.dims()
data.table::setDTthreads(threads)

# Load

## bam2feat.py

In [4]:
# bam2feat.py tables
D = file.path(work_dir, 'features_bam2feat.py')
F = file.path(D, 'feature_files.tsv')
files = Fread(F) 
files

richness,rep,read_depth,assembler,feature_file,MGSIM_reads_params
<dbl>,<int>,<int>,<chr>,<chr>,<chr>
0.8,1,1000000,metaspades,0.8/1/1000000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
0.8,1,1000000,megahit,0.8/1/1000000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
⋮,⋮,⋮,⋮,⋮,⋮
0.8,3,1000000,metaspades,0.8/3/1000000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
0.8,3,1000000,megahit,0.8/3/1000000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712


In [5]:
read_feat = function(x, D){
    df = Fread(file.path(D, x['feature_file']))
    df$richness = x['richness']
    df$rep = x['rep']
    df$read_depth = x['read_depth']
    df$assembler = x['assembler']
    return(df)
}

# loading feature tables
bam2feat_py = files %>%
    apply(1, read_feat, D=file.path(work_dir, 'features_bam2feat.py')) %>%
    rbindlist(use.names=TRUE)
bam2feat_py

assembler,contig,position,ref_base,num_query_A,num_query_C,num_query_G,num_query_T,num_SNPs,coverage,⋯,num_supplementary_SNP,num_secondary_SNP,num_discordant_SNP,seq_window_entropy,seq_window_perc_gc,Extensive_misassembly,Extensive_misassembly_by_pos,richness,rep,read_depth
<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>
metaspades,NODE_2574_length_1001_cov_1.398520,0,C,0,1,0,0,0,1,⋯,0,0,0,1.918,0.5,0,None,0.8,1,1000000
metaspades,NODE_2574_length_1001_cov_1.398520,1,T,0,0,0,1,0,1,⋯,0,0,0,1.918,0.5,0,None,0.8,1,1000000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
megahit,k141_3569,1615,C,0,1,0,0,0,1,⋯,0,0,0,1.459,0.5,0,None,0.8,3,1000000
megahit,k141_3569,1616,C,0,1,0,0,0,1,⋯,0,0,0,1.459,0.5,0,None,0.8,3,1000000


## bam2feat

In [6]:
# bam2feat tables
D = file.path(work_dir, 'features_bam2feat')
F = file.path(D, 'feature_files.tsv')
files = Fread(F) 
files

richness,rep,read_depth,assembler,feature_file,MGSIM_reads_params
<dbl>,<int>,<int>,<chr>,<chr>,<chr>
0.8,1,1000000,metaspades,0.8/1/1000000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
0.8,1,1000000,megahit,0.8/1/1000000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
⋮,⋮,⋮,⋮,⋮,⋮
0.8,3,1000000,metaspades,0.8/3/1000000/metaspades/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712
0.8,3,1000000,megahit,0.8/3/1000000/megahit/features.tsv.gz,--art-paired --art-mflen 250 --art-sdev 10 --rndSeed 8712


In [7]:
# loading feature tables
bam2feat = files %>%
    apply(1, read_feat, D=file.path(work_dir, 'features_bam2feat')) %>%
    rbindlist(use.names=TRUE)
bam2feat

assembler,contig,position,ref_base,num_query_A,num_query_C,num_query_G,num_query_T,num_SNPs,coverage,⋯,num_supplementary_SNP,num_secondary_SNP,num_discordant_SNP,seq_window_entropy,seq_window_perc_gc,Extensive_misassembly,Extensive_misassembly_by_pos,richness,rep,read_depth
<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>
metaspades,NODE_1938_length_1127_cov_2.036381,0,A,1,0,0,0,0,1,⋯,0,0,0,1.250,0.833,0,None,0.8,1,1000000
metaspades,NODE_1938_length_1127_cov_2.036381,1,C,0,1,0,0,0,1,⋯,0,0,0,0.918,1.000,0,None,0.8,1,1000000
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
megahit,k141_12451,1684,T,0,0,0,1,0,1,⋯,0,0,0,1.46,0.500,0,None,0.8,3,1000000
megahit,k141_12451,1685,C,0,1,0,0,0,1,⋯,0,0,0,1.46,0.667,0,None,0.8,3,1000000


# Compare

## Check that dataset matches

In [8]:
bam2feat_py$assembler %>% unique_n('assemblers')
bam2feat$assembler %>% unique_n('assemblers')

No. of unique assemblers: 2 
No. of unique assemblers: 2 


In [9]:
bam2feat_py$contig %>% unique_n('contigs')
bam2feat$contig %>% unique_n('contigs')

No. of unique contigs: 8955 
No. of unique contigs: 8955 


In [10]:
bam2feat_py %>%
    distinct.(assembler, richness, rep, read_depth)

assembler,richness,rep,read_depth
<chr>,<chr>,<chr>,<chr>
metaspades,0.8,1,1000000
megahit,0.8,1,1000000
⋮,⋮,⋮,⋮
metaspades,0.8,3,1000000
megahit,0.8,3,1000000


In [11]:
bam2feat %>%
    distinct.(assembler, richness, rep, read_depth)

assembler,richness,rep,read_depth
<chr>,<chr>,<chr>,<chr>
metaspades,0.8,1,1000000
megahit,0.8,1,1000000
⋮,⋮,⋮,⋮
metaspades,0.8,3,1000000
megahit,0.8,3,1000000


In [12]:
summary_x(bam2feat_py$position)
summary_x(bam2feat$position)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
position,0,1324,21314,83480.16,96603,869434,340906.6,139174.5


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
position,0,1324,21314,83480.16,96603,869434,340906.6,139174.5


## Features

### ref bases

In [13]:
bam2feat_py$ref_base %>% table
bam2feat$ref_base %>% table

.
       A        C        G        T 
15552392 23147449 23200957 15595691 

.
       A        C        G        T 
15552392 23147449 23200957 15595691 

In [14]:
summary_x(bam2feat_py$num_query_A)
summary_x(bam2feat$num_query_A)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_A,0,0,0,5.363889,0,503,204.922,83.659


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_A,0,0,0,5.363889,0,503,204.922,83.659


In [15]:
summary_x(bam2feat_py$num_query_C)
summary_x(bam2feat$num_query_C)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_C,0,0,0,5.073467,3,509,207.15,84.568


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_C,0,0,0,5.073467,3,509,207.15,84.568


In [16]:
summary_x(bam2feat_py$num_query_G)
summary_x(bam2feat$num_query_G)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_G,0,0,0,5.119124,3,504,205.105,83.734


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_G,0,0,0,5.119124,3,504,205.105,83.734


In [17]:
summary_x(bam2feat_py$num_query_T)
summary_x(bam2feat$num_query_T)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_T,0,0,0,5.379804,0,503,204.921,83.659


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_query_T,0,0,0,5.379804,0,503,204.921,83.659


### coverage/SNPs

In [18]:
summary_x(bam2feat_py$coverage)
summary_x(bam2feat$coverage)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
coverage,0,5,12,20.93628,42,509,201.814,82.39


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
coverage,0,5,12,20.93628,42,509,201.814,82.39


In [19]:
summary_x(bam2feat_py$num_SNPs)
summary_x(bam2feat$num_SNPs)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_SNPs,0,0,0,0.01823663,0,324,132.271,53.999


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_SNPs,0,0,0,0.01823663,0,324,132.271,53.999


In [20]:
summary_x(bam2feat_py$num_discordant)
summary_x(bam2feat$num_discordant)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant,0,0,0,0.01239075,0,125,51.03,20.833


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant,0,0,0,0.01239075,0,125,51.03,20.833


### Match features

In [21]:
summary_x(bam2feat_py$min_insert_size_Match)
summary_x(bam2feat$min_insert_size_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_insert_size_Match,0,229,234,234.0048,241,1452,28739,10722.47,4052.711


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_insert_size_Match,0,229,234,234.0274,241,1458,26048,9706.216,3668.605


In [22]:
summary_x(bam2feat_py$mean_insert_size_Match)
summary_x(bam2feat$mean_insert_size_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_insert_size_Match,0,247.5,249.5,249.0798,251.7,1453.5,28739,10718.49,4051.206


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_insert_size_Match,0,247.46,249.54,249.0865,251.67,1458,26048,9702.291,3667.121


In [23]:
summary_x(bam2feat_py$stdev_insert_size_Match)
summary_x(bam2feat$stdev_insert_size_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_insert_size_Match,0,7.8,9.5,9.760158,10.9,848.5,3552204,1342551,507436.6


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_insert_size_Match,0,7.79,9.47,9.747371,10.87,851.36,3552130,1342523,507426


In [24]:
summary_x(bam2feat_py$max_insert_size_Match)
summary_x(bam2feat$max_insert_size_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_insert_size_Match,0,259,265,264.2887,271,1472,28739,10713.6,4049.36


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_insert_size_Match,0,259,265,264.2895,271,1472,26048,9697.624,3665.357


In [25]:
summary_x(bam2feat_py$num_orphans_Match)
summary_x(bam2feat$num_orphans_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_Match,0,0,0,0.0130876,0,61,24.902,10.166


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_Match,0,0,0,0.01285231,0,61,24.902,10.166


In [26]:
summary_x(bam2feat_py$min_mapq_Match)
summary_x(bam2feat$min_mapq_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_mapq_Match,0,42,42,41.82014,42,42,28739,10849.11,4100.58


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_mapq_Match,0,42,42,41.82501,42,42,26048,9832.013,3716.152


In [27]:
summary_x(bam2feat_py$mean_mapq_Match)
summary_x(bam2feat$mean_mapq_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_mapq_Match,0,42,42,41.94408,42,42,28739,10849.11,4100.577


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_mapq_Match,0,42,42,41.94534,42,42,26048,9832.006,3716.149


In [28]:
summary_x(bam2feat_py$stdev_mapq_Match)
summary_x(bam2feat$stdev_mapq_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_mapq_Match,0,0,0,0.06682801,0,29.7,3552204,1342605,507457


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_mapq_Match,0,0,0,0.06476244,0,29.7,3552130,1342577,507446.4


In [29]:
summary_x(bam2feat_py$max_mapq_Match)
summary_x(bam2feat$max_mapq_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_mapq_Match,0,42,42,41.98932,42,42,28739,10849.1,4100.576


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_mapq_Match,0,42,42,41.98937,42,42,26048,9832.003,3716.148


In [30]:
summary_x(bam2feat_py$num_proper_Match)
summary_x(bam2feat$num_proper_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_proper_Match,0,5,12,20.9903,42,455,179.831,73.416


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_proper_Match,0,5,12,20.99008,42,455,179.831,73.416


In [31]:
summary_x(bam2feat_py$num_diff_strand_Match)
summary_x(bam2feat$num_diff_strand_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_diff_strand_Match,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_diff_strand_Match,0,0,0,0,0,0,0,0


In [32]:
summary_x(bam2feat_py$num_orphans_Match)
summary_x(bam2feat$num_orphans_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_Match,0,0,0,0.0130876,0,61,24.902,10.166


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_Match,0,0,0,0.01285231,0,61,24.902,10.166


In [33]:
summary_x(bam2feat_py$num_supplementary_Match)
summary_x(bam2feat$num_supplementary_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_supplementary_Match,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_supplementary_Match,0,0,0,0,0,0,0,0


In [34]:
summary_x(bam2feat_py$num_secondary_Match)
summary_x(bam2feat$num_secondary_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_secondary_Match,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_secondary_Match,0,0,0,0,0,0,0,0


In [35]:
summary_x(bam2feat_py$num_discordant_Match)
summary_x(bam2feat$num_discordant_Match)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant_Match,0,0,0,0.01239075,0,125,51.03,20.833


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant_Match,0,0,0,0.01222659,0,125,51.03,20.833


### SNP features

In [36]:
summary_x(bam2feat_py$min_insert_size_SNP)
summary_x(bam2feat$min_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_insert_size_SNP,136,243,250,249.6806,256,328,74945265,28326555,10706432


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_insert_size_SNP,0,242,249,248.0784,256,1472,74435271,28133733,10633551


In [37]:
summary_x(bam2feat_py$mean_insert_size_SNP)
summary_x(bam2feat$mean_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_insert_size_SNP,136,243,250,249.8978,257,328,74945265,28326555,10706432


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_insert_size_SNP,0,243,249,248.5094,256,1472,74435271,28133732,10633551


In [38]:
summary_x(bam2feat_py$stdev_insert_size_SNP)
summary_x(bam2feat$stdev_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_insert_size_SNP,0,0,0,1.185726,0,82.3,76899114,29065128,10985586


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_insert_size_SNP,0,2.83,7.07,11.62045,12.02,853.48,77361375,29239795,11051604


In [39]:
summary_x(bam2feat_py$max_insert_size_SNP)
summary_x(bam2feat$max_insert_size_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_insert_size_SNP,136,243,250,250.127,257,337,74945265,28326555,10706431


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_insert_size_SNP,0,243,250,248.9159,256,1472,74435271,28133732,10633551


In [40]:
summary_x(bam2feat_py$min_mapq_SNP)
summary_x(bam2feat$min_mapq_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_mapq_SNP,1,42,42,41.85953,42,42,74945265,28326634,10706461


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
min_mapq_SNP,0,42,42,41.71466,42,42,74435271,28133875,10633605


In [41]:
summary_x(bam2feat_py$mean_mapq_SNP)
summary_x(bam2feat$mean_mapq_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_mapq_SNP,1,42,42,41.87337,42,42,74945265,28326634,10706461


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
mean_mapq_SNP,0,42,42,41.73628,42,42,74435271,28133875,10633605


In [42]:
summary_x(bam2feat_py$stdev_mapq_SNP)
summary_x(bam2feat$stdev_mapq_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_mapq_SNP,0,0,0,0.06168604,0,29,76899114,29065131,10985587


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
stdev_mapq_SNP,0,0,0,0.5307096,0,29.7,77361375,29239849,11051624


In [43]:
summary_x(bam2feat_py$max_mapq_SNP)
summary_x(bam2feat$max_mapq_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_mapq_SNP,1,42,42,41.88892,42,42,74945265,28326634,10706461


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,NA's,sd,sd_err_of_mean
max_mapq_SNP,0,42,42,41.76204,42,42,74435271,28133875,10633605


In [44]:
summary_x(bam2feat_py$num_proper_SNP)
summary_x(bam2feat$num_proper_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_proper_SNP,0,0,0,0.04165136,0,286,116.756,47.665


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_proper_SNP,0,0,0,0.0418745,0,302,123.288,50.332


In [45]:
summary_x(bam2feat_py$num_diff_strand_SNP)
summary_x(bam2feat$num_diff_strand_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_diff_strand_SNP,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_diff_strand_SNP,0,0,0,0,0,0,0,0


In [46]:
summary_x(bam2feat_py$num_orphans_SNP)
summary_x(bam2feat$num_orphans_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_SNP,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_orphans_SNP,0,0,0,0.0002352881,0,33,13.472,5.5


In [47]:
summary_x(bam2feat_py$num_supplementary_SNP)
summary_x(bam2feat$num_supplementary_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_supplementary_SNP,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_supplementary_SNP,0,0,0,0,0,0,0,0


In [48]:
summary_x(bam2feat_py$num_secondary_SNP)
summary_x(bam2feat$num_secondary_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_secondary_SNP,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_secondary_SNP,0,0,0,0,0,0,0,0


In [49]:
summary_x(bam2feat_py$num_discordant_SNP)
summary_x(bam2feat$num_discordant_SNP)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant_SNP,0,0,0,0,0,0,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
num_discordant_SNP,0,0,0,0.0001641623,0,52,21.229,8.667


### G+C & entropy

In [50]:
summary_x(bam2feat_py$seq_window_entropy)
summary_x(bam2feat$seq_window_entropy)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq_window_entropy,0,1.459,1.459,1.493931,1.792,1.918,0.691,0.282


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq_window_entropy,0,1.46,1.46,1.493877,1.79,1.92,0.691,0.282


In [51]:
summary_x(bam2feat_py$seq_window_perc_gc)
summary_x(bam2feat$seq_window_perc_gc)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq_window_perc_gc,0,0.5,0.667,0.5980895,0.833,1,0.343,0.14


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq_window_perc_gc,0,0.5,0.667,0.5980895,0.833,1,0.343,0.14


# sessionInfo

In [52]:
sessionInfo()

R version 4.1.0 (2021-05-18)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Anxiety_Twins_Metagenomes/envs/tidyverse2/lib/libopenblasp-r0.3.12.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.9 tidytable_0.6.3   data.table_1.14.0 ggplot2_3.3.5    
[5] tidyr_1.1.3       dplyr_1.0.7      

loaded via a namespace (and not attached):
 [1] pillar_1.6.1      compiler_4.1.0    R.methodsS3_1.8.1 base64enc_0.1-3  
 [5] R.utils_2.10.1    tools_4.1.0       